In [ ]:
from IPython.display import display, Javascript, HTML
import ipywidgets as ipw
from src import widgets, utils
import os

In [ ]:
CONFIG_FILENAME = "schema/config.json"
CONFIG = utils.read_json(CONFIG_FILENAME)
CONFIG_ELN = utils.get_aiidalab_eln_config()
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

In [ ]:
if OPENBIS_SESSION:
    increase_buttons_size = HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
    drafts_vbox = widgets.CreateDraftsWidget(OPENBIS_SESSION)

home_button = ipw.Button(
    description = '', 
    disabled = False, 
    button_style = '', 
    tooltip = 'Main menu', 
    icon = 'home', 
    layout = ipw.Layout(width = '100px', height = '50px')
)

save_button = ipw.Button(
    description = '', 
    disabled = False, 
    button_style = '', 
    tooltip = 'Save', 
    icon = 'save', 
    layout = ipw.Layout(width = '100px', height = '50px')
)

save_close_buttons_hbox = ipw.HBox([save_button, home_button])

In [ ]:
def close_notebook(b):
    display(Javascript(data = 'window.location.replace("home.ipynb")'))

def save_drafts(b):
    project_id = drafts_vbox.select_project_widget.project_dropdown.value
    if project_id == "-1":
        return
    else:
        project_collections = utils.get_openbis_collections(OPENBIS_SESSION, project = project_id)
        drafts_collection = None
        for collection in project_collections:
            if collection.props["name"] == "Drafts":
                drafts_collection = collection
        
        if drafts_collection is None:
            drafts_collection = utils.create_openbis_collection(
                OPENBIS_SESSION, 
                type = "COLLECTION",
                project = project_id,
                code = "DRAFTS_COLLECTION",
                props = {"name": "Drafts"}
            )
            
        results_ids = list(drafts_vbox.select_results_selector.value)
        draft_parents = results_ids
        draft_type = "DRAFT"
        draft_type_lower = draft_type.lower()
        
        draft_properties = {
            "name": drafts_vbox.name_textbox.value,
            f"{draft_type_lower}.draft_type": drafts_vbox.draft_type_dropdown.value,
            "description": drafts_vbox.description_textbox.value,
            "comments": drafts_vbox.comments_textbox.value
        }
        
        draft_object = utils.create_openbis_object(
            OPENBIS_SESSION,
            type = draft_type,
            parents = draft_parents,
            collection = drafts_collection,
            props = draft_properties
        )
        
        utils.upload_datasets(
            OPENBIS_SESSION, 
            draft_object, 
            drafts_vbox.support_files_uploader, 
            "ATTACHMENT"
        )
        
        # Reset widgets
        drafts_vbox.select_project_widget.project_dropdown.value = "-1"

# Create draft

In [ ]:
if OPENBIS_SESSION:
    display(drafts_vbox)
    display(increase_buttons_size)

home_button.on_click(close_notebook)
save_button.on_click(save_drafts)
display(save_close_buttons_hbox)